# Project: Disaster Relief
#Section 2: Data Preprocessing and Simple Models


Today we will be building some more sophisticated models called One-hot-encoding, CountVectorizer, and Bag of Words. You can review the slides to remember how these models work. But first we will need to clean our data a little bit, an important step in any AI Pipeline.

In this notebook we'll be:
*   Preprocessing the Dataset for ML
*   Implementing a Bag of Words Vectorizer
*   Implementing Logistic Regression ML Models



In [ ]:
#@title Load your dataset { display-mode: "form" }
# Run this every time you open the spreadsheet
%load_ext autoreload
%autoreload 2
from collections import Counter
from random import sample
from importlib.machinery import SourceFileLoader
import numpy as np
from os.path import join
import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('punkt')
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download('stopwords' ,quiet=True)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import multilabel_confusion_matrix
from sklearn import metrics


!wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Disaster%20Relief/disaster_data.csv'
dataset_path = './disaster_data.csv'

import nltk
import string
nltk.download('stopwords')


nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


disaster_data.csv   100%[===================>] 222.42K  --.-KB/s    in 0.003s  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#@title If the previous cell fails to load data, use this cell
import re
import seaborn as sns
import pandas as pd
import numpy as np
from torchtext.vocab import GloVe
from sklearn.model_selection import train_test_split



from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
import requests, io, zipfile

ModuleNotFoundError: No module named 'torchtext'

# Introduction

Up until now, we have looked at disaster-related tweets, made a rule-based classifier, and evaluated it. Today, we shall clean the data, stem it, and learn about One-Hot Encoding, CountVectorizer, and Logistic Regression.

Before, we do all this, we must understand what a model really is.

What is a model? A model is something that we will make to predict the category of a given tweet. It is a numerical understanding of the data such that given a new data point, we can figure out how it links to the previous data. This is a crude definition, which you will understand more through this project.

The models we have used or will use in this project are:

1. Rule Based Classifier (You specify rules based on which the model gives the output - the category of the tweet)
2. CountVectorizer + Logistic Regression: A model based on counting the number of occurances of a word and applying regression on it to predict the category of a new tweet
3. Word2Vec + Logistic Regression: A model which applies the idea that words that occur in similar contexts tend to be close in a sentence, and uses this to predict the category of the tweet.


## Data Preprocessing

Before we can jump into building a model, we must clean the data a little bit!

In [ ]:
# Load the data.
disaster_tweets = pd.read_csv('disaster_data.csv',encoding ="ISO-8859-1")

**Exercise:** Show the top five tweets in our `disaster_tweets` dataset.

In [ ]:
### Your code here
disaster_tweets.head()

**Discussion Exercise**: Consider the tweet *I really need food.... I am very hungry. The hunger is unbearable. #pleasehelp*

1. Are all words in the tweet equally informative?
2. Are there any words in this sentence that mean the same thing, but are technically distinct words?
3. Are there any unnecessary words or symbols that we could remove from the tweet before building a model?

We are going to play with three pre-processing steps to address these two questions.

###Removal of non alphabetic characters

In tweet classification we use words as the features, so it's important to remove unwanted characters such as numbers and punctuation marks as they don't provide us with any valuable information.

To do this, we'll use **regular expressions**. Regular expressions are a powerful tool that perform *pattern matching*, allowing us to search a piece of text for patterns and replace all occurrences of a pattern with something else.

For example, suppose we want to remove all occurrences of the letter "X" from a block of text (for whatever reason). This is the same as replacing the letter "X" with the empty string "". We can use the function `re.sub` to do this in Python:

> `text = re.sub(r"X", "", text)`

Below, we use `tweets = tweets.apply(lambda x: re.sub(r'[^a-zA-Z0-9]+', ' ',x))` to use regular expressions on every tweet in the `tweets` data.

What is our `re.sub` function replacing?

In [ ]:
#Read the tweet data and convert it to lowercase
tweets = disaster_tweets['text'].str.lower()
tweets = tweets.apply(lambda x: re.sub(r'[^a-zA-Z0-9]+', ' ',x))

In [ ]:
#Extract the labels from the csv
tweet_labels = disaster_tweets['category']

### Tokenizing
First we need to split a sentence into individual words, or *tokens*.


###**Discussion Exercise**:

What would be the token list be of the following sentence be? "*AI is so fun! I love to learn about NLP and machine learning!*""

Discuss your answer with your group, then write it down in your worksheet.

In [ ]:
#@title Tokenizing
tweet = "AI is so fun! I love to learn about NLP and machine learning! Enter your own tweet here" #@param {type:'string'}
for i in word_tokenize(tweet):
  print(i)

## Stemming and Lemmatization

Remember that the goal of both stemming and lemmatization is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form.

A difference between stemming and lemmatization is that stemming looks at the current word only, while lemmatization also takes the context into consideration. Either way, this pre-processing step could be somewhat tedious. Luckily, the powerful `nltk`, (short for the [National Language Toolkit](https://www.nltk.org/) library), provides tools for both.



### Exercise: Stemming using the Porter stemmer
*Porter's algorithm*, developed in the 1980s, is one of the most commonly used stemmers.


Try and find a word that Porter's stemming doesn't work well on! (Hint: Try some plurals of words that end in -e)

What happens when you input multiple plural words?

In [ ]:
#@title Stem words { run: "auto", vertical-output: true, display-mode: "form" }
stemmer = PorterStemmer()
word = "leaves" #@param {type:"string"}
print(stemmer.stem(word))


### Lemmatizer

With the stemmer, you might have found that the results may look a bit mechanical. This is because the Porter's algorithm is essentially a sequential application of a set of rules. To get better looking results, let's try out a lemmatizer.

In [ ]:
#@title Lemmatize Words { run: "auto", vertical-output: true, display-mode: "form" }
# Get the lemmatizer
nltk.download('omw-1.4')
lemma = WordNetLemmatizer()
word = "leaves" #@param {type:"string"}
print(lemma.lemmatize(word))

###**Discussion Exercise**:

**Discuss:**
* What are the differences between the Porter stemmer and the lemmatizer?
* How do you think the lemmatizer works?

## Stop Words

###**Discussion Exercise**:

In [ ]:
#@title Are there words that can be removed without affecting the model?

#@markdown Hint: Which words can we remove without changing the meaning of a sentence?
words = "so" #@param {type:"string"}

Stop words are words that occur in both categories, that are not relevant to the context, such as 'at', 'is', 'the' and so on... It is usually advantageous for the classifier to ignore these stop words, since they may add noises or cause numerical issues as they add baggage to the model

In [ ]:
#@title Run this cell for some example stop words! { vertical-output: true, display-mode: "form" }
#@markdown *Your groupmates will likely have different examples.*
eng_stopwords = stopwords.words('english')
for i, word in enumerate(sample(eng_stopwords, 10)):
    print(word)

Let us see if the words you identified are stop words or not. Input them down below to see!

In [ ]:
#@title Check stop words { run: "auto", vertical-output: true, display-mode: "form" }
word = "inside" #@param {type:"string"}
if not word: raise Exception('Please enter a word')
eng_stopwords = set(stopwords.words('english'))
if word.lower().strip() in eng_stopwords: print('YES')
else: print('NO')



## Preprocessing pipeline of our data

Explore how combining these methods changes the structure of our tweet dataset. Change out the `None` values to complete our `remove_stopwords()` function!

In [ ]:
stopword_set = set(stopwords.words('english'))

# Complete the following function to remove the stopwords from the tokenized tweets
def remove_stopwords(token_list):
  filtered_sentences = []
  ### YOUR CODE HERE
  for tweet in token_list:
    new_tweet = []
    for word in tweet:
      if word not in stopword_set:
        new_tweet.append(word)
    filtered_sentences.append(new_tweet)
  ### END CODE
  return filtered_sentences

Remember the cell we ran at the top of this notebook to import our data? Inside, we imported the helper function `word_tokenize` from `nltk`. Below, we use `word_tokenize` to tokenize every tweet in our `tweets` data.

**Exercise:** Use the `remove_stopwords()` function we defined above to remove the stopwords in our `tokenized_tweets` data and print the first five!

In [ ]:
# Tokenize all the tweets
tokenized_tweets = [word_tokenize(t) for t in tweets]

### YOUR CODE HERE
tweet_set = remove_stopwords(tokenized_tweets)
# Remove stopwords from all the tweets
for i in range(5):
  print("Original tweet: %s: \nCleaned and tokenized data: %s\n" % (tweets[i], tweet_set[i]))

### END CODE


# Bag of Words Model

## One-Hot Encoding
**One-Hot Encoding**, also known as one-of-K scheme is a way to encode the data to be used in other functions (such as linear regression).

Let us consider an example to understand one-hot encoding:

Before we apply a model on our tweets, we need to convert it to a form the model, i.e. a machine, can understand - essentially convert a tweet to numerical form. We cannot just pass words to the model, because it won't know what those mean and might try and extract information from them. Hence, a numerical format is the best.

The easiest way to do so is to map each word in a tweet to a number, a categorical value. This will represent all words in a tweet uniquely!

---
Suppose we have a tweet:
> Tweet: 'I am hungry need food' <br>
> Category: Food

Its numerical representation would be:

In [ ]:
#@title Run this cell to see the numeric representation! { vertical-output: true, display-mode: "form" }
d = {'I': 1, 'am': 2, 'hungry': 3, 'need': 4, 'food': 5}
print('{:<12}|{:>2}'.format('word', 'value'))
print('-------------------')
for k,v in d.items(): print('{:<12}|{:>3}'.format(k,v))

**Discussion Exercise**: Why do you think the above representation is wrong? What information could a model possibly extract from the above information such that its conclusions would be wrong/way off to what we want to achieve?


We need to encode the words and include them as a feature to train the model. That is where One-Hot Encoding comes into play. Understanding how it is done will make the process clearer.

Let us consider the same example and see what its one-hot encoding would be:

In [ ]:
#@title Run this cell to see the One-Hot Encoding! { vertical-output: true, display-mode: "form" }
print('{:^10}|{:^7}|{:^8}|{:^2}|{:^9}'.format('I', 'am', 'hungry','need','food'))
print('---------------------------------------------------')
print('{:^10}|{:^7}|{:^8}|{:^4}|{:^9}'.format('1', '0', '0','0','0'))
print('{:^10}|{:^7}|{:^8}|{:^4}|{:^9}'.format('0', '1', '0','0','0'))
print('{:^10}|{:^7}|{:^8}|{:^4}|{:^9}'.format('0', '0', '1','0','0'))
print('{:^10}|{:^7}|{:^8}|{:^4}|{:^9}'.format('0', '0', '0','1','0'))
print('{:^10}|{:^7}|{:^8}|{:^4}|{:^9}'.format('0', '0', '0','0','1'))


Above, each letter is represented using a row of 1's and 0's, a row which can essentially represent the whole of the vocabulary. This is called one-hot encoding.

In [ ]:
#@title (Optional): What are the encodings for the inputs below?
i = "[1, 0, 0, 0, 0]" #@param ["Choose an answer", "[1, 0, 0, 0, 0]", "[1, 1, 0, 0, 1]", "[1, 0, 0, 1, 1]", "[1, 0, 1, 0, 0]", "[0, 1, 0, 1, 1]"]
i_hungry = "[1, 0, 1, 0, 0]" #@param ["Choose an answer", "[1, 0, 0, 0, 0]", "[1, 1, 0, 0, 1]", "[1, 0, 0, 1, 1]", "[1, 0, 1, 0, 0]", "[0, 1, 0, 1, 1]"]
i_need_food = "[1, 0, 0, 1, 1]" #@param ["Choose an answer", "[1, 0, 0, 0, 0]", "[1, 1, 0, 0, 1]", "[1, 0, 0, 1, 1]", "[1, 0, 1, 0, 0]", "[0, 1, 0, 1, 1]"]

answers = [i, i_hungry, i_need_food]
words = ["i", "i_hungry", "i_need_food"]
encodings = ["[1, 0, 0, 0, 0]",
           "[1, 0, 1, 0, 0]",
           "[1, 0, 0, 1, 1]"]

for i, actual in enumerate(encodings):
  if actual == answers[i]:
    print("Yes, that's the encoding for '{}'!".format(words[i]))
  else:
    print("No, that's not the encoding for '{}'.".format(words[i]))


## **Terms to Know:**

Before we code our One-Hot Encoding function, let's get to know some new data structures we'll be seeing.

### Sets

A set is an unordered collection of unique elements.

You can create sets like this:
```
integer_set = {5, 6, 7, 8}
string_set = {"hello", "world"}
```
You can also turn a list into a set using `set()`.

In [ ]:
my_list = [1, 1, 2, 3, 4]
list_to_set = set(my_list)

print(list_to_set)

Notice that `my_list` had two 1's, and `list_to_set` only has one. Sets do not contain duplicate elements.

### Dictionaries

A dictionary is an unordered data structure containing key-value pairs, where each key maps to its value.

You can create dictionaries like this:
```
grocery_dict = {1: "bread", 2: "bananas", 3: "milk"}
pet_dict = {"cats": 5, "dogs": 3, "rabbits": 2}
```
Let's use `pet_dict` as an example. The key `cats` points to the value `5`. Run the cell below to see how we can access a value from a dictionary.

Try to access other values!

In [ ]:
pet_dict = {"cats": 5, "dogs": 3, "rabbits": 2}

num_cats = pet_dict["rabbits"] # Change this!
print(num_cats)

We can also change the value in a key-value pair:

In [ ]:
pet_dict["cats"] = 0 # Change this!

print(pet_dict)

Here's how we might check if a key is in a dictionary and add it if not:

In [ ]:
pet = "lizard" # Change this!

if pet in pet_dict:
  print("Yes, our pet_dict contains one or more", pet)
else:
  pet_dict[pet] = 1 # Change this!

print(pet_dict)

Feel free to revisit the above section when needed. Sets and dictionaries are useful data structures!

### One-Hot Encoding Function

**Exercise:** Complete the `one_hot_encoding` function below!

**Note:** You'll use the function `enumerate(list)`. We can use `enumerate()` to keep track of elements of a list passed in, and their respective indices `idx`. Read more about `enumerate()` [here](https://realpython.com/python-enumerate/)!

In [ ]:
def one_hot_encoding(sentences, sentence, print_word_dict = False):
  """
  param: sentences - list of sentences to form the one-hot encoding
  param: sentence - sentence to return the one-hot encoding of
  return: sent_encoding - the encoded sentence
  """

  words_list = [] # an empty list

  ### YOUR CODE HERE: Change the None values!

  # Step 1: Tokenize each sentence using word_tokenize() and add it to the list
  for sent in sentences:
    words_list.extend(word_tokenize(sent))

  # Step 2: Remove the duplicates in your words_list
  words_list = set(words_list) # Hint: See Sets above!

  # Step 3: Make an empty dictionary
  words_map_dict = {}

  # Step 4:
  # Add every word in your words_list as a key to your dictionary
  # Pair each word with its index as the value
  for idx, w in enumerate(words_list):
    words_map_dict[w] = idx

  # a numpy array of all zeros of the same length as words_list
  sent_encoding = np.zeros(len(words_list)) # Our future one-hot encoding!

  # Step 5:
  # Loop through the words in the given sentence (sentence to check)
  # Find the index of each word using our dictionary
  # Increment the value of the numpy array at that index by one
  for word in word_tokenize(sentence):
    if word in words_map_dict:
      print(word)
      idx = words_map_dict[word]
      sent_encoding[idx] += 1

  ### END CODE

  if print_word_dict:
    print ("Word Dictionary: ", words_map_dict)

  return sent_encoding

In [ ]:
encoding = one_hot_encoding(["oh I love AI", "AI is so much fun", "is AI fun or what"], "oh what fun AI is ", True)
print("Encoding: ", encoding)

## Count Vectorizer - The Bag of Words Model

### Introduction

Now that we have the one-hot encoding for all tweets, it's time to attach value to each word, such that it can be distinguished and used by the model.

One idea is to assign each word the same weight, say a weight of one. However, that is non distinguishable and the model won't be able to learn anything with that information. Can you think of a simple way to add weights to words which, say, occur more frequently?

That is the Bag of Words Model. The **Bag of Words Model** converts the tweets into a matrix of token counts. Let us consider an example.

Recall the first 3 tweets:

In [ ]:
for t in tweets[:3]:
  print(t)

The Vocabulary for the tweets would be:

In [ ]:
#@title Vocabulary { vertical-output: true, display-mode: "form" }
word_count = Counter()
for tweet in tweets[:3]:
  for t in word_tokenize(tweet):
    word_count[t]+=1
word_count_list = [(k,v) for k,v in word_count.items()]
word_count_list.sort(key=lambda x:x[0])
print('{:<12}|{:>2}'.format('word', 'position'))
print('-------------------')
for k,v in enumerate(word_count_list): print('{:<12}|{:>3}'.format(v[0],k))

If there are N words in the vocabulary, each row in the matrix would be of N words.

Based on this, one-hot encoding for each tweet and the matrix that will be given to the model would be the following.


In [ ]:
print("Tweet 1: ", one_hot_encoding([str(t) for t in tweets[:3]], str(tweets[0])))
print("Tweet 2: ", one_hot_encoding([str(t) for t in tweets[:3]], str(tweets[1])))
print("Tweet 3: ", one_hot_encoding([str(t) for t in tweets[:3]], str(tweets[2])))

In [ ]:
#@title Token Counts { vertical-output: true }
print('{:<12}|{:>2}'.format('word', 'word_count'))
print('-------------------')
for k,v in word_count_list: print('{:<12}|{:>3}'.format(k,v))

The above is the Bag of Words model. Sklearn's `Countvectorizer` does the same thing, however, in a much more sophisticated manner.

In [ ]:
train_text = [t for t in tweets[:3]]
print(train_text)
vectorizer = CountVectorizer()
vectorizer.fit(train_text)
print('Number of Words in Vocabulary of train tweets are: {}'.format(len(vectorizer.vocabulary_)))

And the vocabulary is *almost* the same as ours above.

**Discussion Exercise**: What is different about vocabulary results from the `CountVectorizer` than our method above?   Why do you think that is? Which is better?

In [ ]:
vectorizer.vocabulary_

### CountVectorizer - Fit and Transform

The `.fit()` method fits the given data to the vectorizer - consider it as if it is learning from the data. It produces a matrix, which will then be passed on to the logistic regression (later, so don't worry about understanding that). Think of the `.fit()` method this way: it just allows you to generate the matrix (but doesn't let us see it) used by the model to learn information about the training data.

The `.transform()` method transforms the given data to the matrix- there is no learning here. It does not generate a vocabulary, nor does it allow for many other functionalities that the `.fit()` method allows for. Hence, it cannot be used as a substitute for the `.fit()` method as it just spews out the matrix representation of the data!<br>

### Coding Exercise

Let us make the `CountVectorizer` and use the `.fit()` and `.transform()` methods to learn their workings. Fill out the code below and try it on tweets of your choice!

In [ ]:
tweet_01 = 'This is a big tweet '
tweet_02 = 'Sample Tweet two'
# You can add more tweets here
train_text = [tweet_01, tweet_02]
print(train_text)

tweet_03 = 'This This sample tweet' # Transform this once the vectorizer is ready

### Your code starts here ###

# Initialize the vectorizer
vectorizer = CountVectorizer()
# Fit the train_text data to the vectorizer
vectorizer.fit(train_text)
# Print the vocabulary of the vectorizer
print(vectorizer.vocabulary_)
# Transform this tweet_03's tokenList
print(vectorizer.transform([tweet_03]))

### Your code ends here ###



# Logistic Regression

## Logistic Regression Refresher

We've just spent the last week or so learning about more sophisticated neural network architectures.  Remember that logistic regression is just linear regression followed by a sigmoid function.

**Review:** What is Logistic Regression?

Logistic regression is a type of linear regression that is generally used for classification. Unlike linear regression which outputs continuous number values, logistic regression uses the logistic function, also called the sigmoid function, to transform the output to return a probability value between 1 and 0, which can then be mapped to the different categories. The logistic (sigmoid) function looks something like this:

![Logistic Function](https://ml-cheatsheet.readthedocs.io/en/latest/_images/sigmoid.png)

Consider an example to understand logistic regression and to enhance the difference between logistic and linear regression:

Given data on time spent studying and exam scores:

>**Linear Regression** could help us predict the student’s test score on a scale of 0 - 100. Linear regression predictions are continuous (numbers in a range).

>**Logistic Regression** could help us predict whether the student passed or failed. Logistic regression predictions are discrete (only specific values or categories are allowed). We can also view probability scores underlying the model’s classifications.

 **Why Logistic Regression?**

**A couple of reasons for using Logistic Regression:**
1.   Using a simpler model tells us how much room we have to improve.
2.   A simple model makes iteration quick and easy.
3.   Lastly, and perhaps most importantly, logistic regression is interpretable. You may have heard in the past that one thing deep neural networks struggle with is interpretability–when you are using these models to make predictions that affect people's wellbeing (e.g., sentencing decisions, predictive policing decisions), it becomes extremely important that you are able to understand why a model is making the predictions it makes.For simpler models like logistic regression, we get interpretability for free!

## Logistic Regression in Python

Logistic regression in python can be done easily with the help of sklearn's `LogisticRegression()` function. Let us first do it for the three tweet examples that we saw above.

In [ ]:
tweet1 = 'please help we desperately need food'
label1 = "Food"
tweet2 = 'We are very thirsty please send water'
label2 = 'Water'
tweet3 = 'we need water and are very thirsty'
label3 = 'Water'

train_tweets = [tweet1, tweet2]
train_tweets_label = [label1, label2]
test_tweets = [tweet3]
test_tweets_label = [label3]

Next, let us make the vectorizer and encode tweets:

In [ ]:
vectorizer = CountVectorizer()
train_vect = vectorizer.fit_transform(train_tweets) # fit_transform fits the train tweets and returns the sparse matrix of the tweets
model = LogisticRegression()
model.fit(train_vect, train_tweets_label) # Fit the data values to the model

Over here, `model.fit(train_vect, train_tweets_label)` applies logistic regression to the data given by the  matrix and hence, fits the data to the function. It takes two arguments: the matrix that is the `train_vect` variable and the `train_tweets_label`, which is the category each tweet belongs to.

Below, we transform the `test_tweets` into a vector form. Now predict the third tweet using this model.

*Hint: What function have we used in the ML-pipeline to test, or **predict** test data?*

In [ ]:
test_vect = vectorizer.transform(test_tweets)
### YOUR CODE HERE: Have your model predict on the third tweet!
result = model.predict(test_vect)
print('Actual Category: {}\nPredicted Category: {}'.format(label3, result[0]))

Yay! It predicted it correctly! However, that might not always be the case, as the training set here is so small.

**Exercise**: Can you trick the logistic regression? Try and make a tweet get classified as "Water" when it is about "Food".  *Hint: What are some words that were in the Food tweet in the training set that had no significance to food?*

In [ ]:
#@title Trick the logistic regression { run: "auto", vertical-output: true, display-mode: "form" }
test_tweet = "" #@param {type:"string"}
true_label = "Water" #@param ["Food", "Water"]

test_vect = vectorizer.transform([test_tweet])
result = model.predict(test_vect)
print('Actual Category: {}\nPredicted Category: {}'.format(true_label, result[0]))


## Logistic Regression for Tweet Classification (Coding Exercise)

In [ ]:
#Split the Data into Training and Testing
X_train, X_test, y_train, y_test = train_test_split(tweet_set, tweet_labels, test_size=0.2, random_state=1)

Let us now build our own regression model for all the training data. Complete the code for `train_model` and `predict` below:

In [ ]:
def train_model(tweets_to_train,train_labels):
  """
  param: tweets_to_train - list of tweets to train on
  return: the vectorizer, the logistic regression model, the train_vector
  """

  train_tweets = [" ".join(t) for t in tweets_to_train]
  train_tweets_label = [l for l in train_labels]

  ### Your code starts here ###
  vectorizer = CountVectorizer() # initialize CountVectorizer
  train_vect = vectorizer.fit_transform(train_tweets)

  model = LogisticRegression() # create LogisticRegression Model
  model.fit(train_vect, train_tweets_label) # train on transformed train data and our labels

  ### Your code ends here ###

  return model, vectorizer

In [ ]:
def predict(tweets_to_test, vectorizer, model):
  """
  param: tweets_to_test - list of tweets to test the model on
  param: vectorizer - the CountVectorizer
  param: model - the LogisticRegression model
  return result (the prediction), the test_vect
  """

  test_tweets = [" ".join(t) for t in  tweets_to_test]
  print(test_tweets)

  ### Your code starts here

  test_vect = vectorizer.transform(test_tweets) # Use .transform to vectorize our tweets
  result = model.predict(test_vect) # Have your model predict on the vectorized tweets

  ### Your code ends here

  return result

Check out our work in action!

In [ ]:
model, train_countvect = train_model(X_train, y_train)

#Predict labels for test set
y_pred = predict (X_test, train_countvect, model)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

# Evaluation
Let's see how our classifier did!  We will train our classifier on 80% of the dataset and then test it on 20%. This is called a *train-test split* and is usually done to evaluate models.

In [ ]:
table=pd.DataFrame([[" ".join(t) for t in X_test],y_pred, y_test]).transpose()
table.columns = ['Tweet', 'Predicted Category', 'True Category']
print("Percent Correct: %.2f" % (sum(table['Predicted Category'] == table['True Category'])/len(table['True Category'])))
table

**Discussion Exercise**: Which categories does the regressor perform best on?  Would the classifier perform better or worse if we only used the food vs water tweets?

## Evaluation Metrics

Let us look at some stats about the prediction to understand what the model predicted! Review day 1 for a refresher on accuracy-related metrics.

In [ ]:
#@title Run this to load the helper function for plotting our confusion matrix!
'''
Plots the confusion Matrix and saves it
'''
def plot_confusion_matrix(y_true,y_predicted):
  cm = metrics.confusion_matrix(y_true, y_predicted)
  print ("Plotting the Confusion Matrix")
  labels = ['Energy', 'Food', 'Medical', 'None', 'Water']
  df_cm = pd.DataFrame(cm,index =labels,columns = labels)
  fig = plt.figure()
  res = sns.heatmap(df_cm, annot=True,cmap='Blues', fmt='g')
  plt.yticks([0.5,1.5,2.5,3.5,4.5], labels,va='center')
  plt.title('Confusion Matrix - TestData')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()
  plt.close()




In [ ]:
plot_confusion_matrix(y_test,y_pred)

Review day 1 for a refresher on accuracy metrics!

In [ ]:
print('The total number of correct predictions are: {}'.format(sum(table['Predicted Category'] == table['True Category'])))
print('The total number of incorrect predictions are: {}'.format(sum(table['Predicted Category'] != table['True Category'])))

In [ ]:
print('Accuracy on the test data is: {:.2f}%'.format(metrics.accuracy_score(y_test, y_pred)*100))

## Conclusion

**Discuss:** How could you use what you have built to help during a disaster?


That is it for today! Next, we'll cover another model - GloVe. Review the concepts we covered today and enjoy the rest of your day!
